All the basic imports.

In [187]:
import pandas as pd                                         #https://pandas.pydata.org/docs/index.html
from pandas import read_csv,read_json

import imageio
import os

Some constants.

In [188]:
path_input='data/img/'
path_output='data/img_processed/'
image_num='01243'

First I will try to work on text recognition in an image.

Pytesseract is an optical character recognition tool for python. It is a wrapper for tor Google's Tesseract-OCR Engine.

In [189]:
import pytesseract                                          #https://pypi.org/project/pytesseract/

print(pytesseract.image_to_string(path_input+image_num+'.png',lang='eng'))

" * te
When your human’ says: “%
_ “who's agood girlas — el

im o->
4 a7

} >
—

and you already know it's you



Shit doesn't work well out of the box.

It seems I will need to check out OpenCV's threshold blah blah.

I'm sleepy will try tomorrow.

Fuck tomorrow, today it is.

In [190]:
import cv2 as cv                                    #https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

from cv2 import threshold,adaptiveThreshold         #threshold needs a grayscale image to work properly      
#thresholding-> https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
#threshold function-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57
#adaptiveThreshold function-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3

from cv2 import cvtColor                            #changes colorspace based on the flag provided             
#colorspace-> https://docs.opencv.org/4.x/df/d9d/tutorial_py_colorspaces.html
#cvtColor function-> https://docs.opencv.org/4.x/d8/d01/group__imgproc__color__conversions.html#ga397ae87e1288a81d2363b61574eb8cab

from cv2 import COLOR_BGR2GRAY                      #imports the flag for BGR to Grayscale conversion
#list of conversion flags-> 

from cv2 import ADAPTIVE_THRESH_MEAN_C,ADAPTIVE_THRESH_GAUSSIAN_C       #imports the flags for adaptive thresholding
#list of adaptive threshold function flags-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gaa42a3e6ef26247da787bf34030ed772c

from cv2 import THRESH_BINARY,THRESH_BINARY_INV     #imports the flags for types of thresholding
#list of thresholding types flags-> https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576

Applying a simple grayscale (B/W) conversion to the image with opencv.

In [191]:
img=cv.imread(path_input+image_num+'.png')
gray=cvtColor(img,COLOR_BGR2GRAY)

if os.path.isdir(path_output):
    cv.imwrite(path_output+image_num+'.png',gray)
else:
    os.mkdir(path_output)
    cv.imwrite(path_output+image_num+'.png',gray)

After the grayscale conversion, tesseract character recognition accuracy has improved (?)

In [192]:
print(pytesseract.image_to_string(path_output+image_num+'.png',lang='eng'))

when your humansays:
_ ‘“who>sa good gira.
Lh « 79

and. you already Know it's you



Time for threshold. After a lot of try and error and many voodoo dolls burned and demons summoned, it works (?)

In [193]:
gray_thres=adaptiveThreshold(gray,255,ADAPTIVE_THRESH_GAUSSIAN_C,THRESH_BINARY,3,9)
cv.imwrite(path_output+image_num+'_t.png',gray_thres)

True

In [194]:
print(pytesseract.image_to_string(path_output+image_num+'_t.png',lang='eng'))

Hie Our IMA Says

“who's a good oi”

and sou alreaaly now i's you




Copied code from G4G gives similar output.

In [195]:
ret, thresh1 = cv.threshold(gray, 0, 255, cv.THRESH_OTSU | cv.THRESH_BINARY_INV)

# Specify structure shape and kernel size. 
# Kernel size increases or decreases the area 
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect 
# each word instead of a sentence.
rect_kernel = cv.getStructuringElement(cv.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv.dilate(thresh1, rect_kernel, iterations = 1)
contours, hierarchy = cv.findContours(dilation, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = gray_thres.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]
    
    # Open the file in append mode
    file = open("recognized.txt", "a")
    
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)
    
    # Appending the text into file
    file.write(text)
    file.write("\n")
    
    # Close the file
    file.close()


OKAY FUCK TESSERACT, ITS SHIT, GONNA TRY KERAS OCR NEXT

In [ ]:
import keras_ocr                                                